In [ ]:
%%bash
cdo selyear,1980/2018 -selname,olr /home/panini/DATA/olr/olr.day.mean.nc olr_1980-2018.nc
cdo ydaymean olr_1980-2018.nc olr_daily_climatology_1980-2017.nc
cdo -b 32 sub olr_1980-2018.nc olr_daily_climatology_1980-2017.nc olr_daily_anomalies_1980-2018.nc

In [1]:
;mjoclivar_6.ncl
load "$NCARG_ROOT/lib/ncarg/nclscripts/csm/diagnostics_cam.ncl"    


    
    ; time window : include enough buffer
  twStrt = 19800101      ; 4 years: winter 96-97 MJO gold standard
  twLast = 20181231

  thStrt = 19960101      ; Hov start time
  thLast = 19971231      ; Hov last  time

  spd    = 1.
  bpf    = (/20,100,201/)

 ;diri    = "/Users/shea/Data/AMWG/"
  diri    = "./"            ; new input directory
; vName   = "olr"           ; name of variable on the file
; fili    = "olr.day.mean.nc"
; vName   = "U_anom"        ; name of variable on the file
; fili    = "uwnd.day.850.anomalies.1980-2005.nc"

  vName   = "olr"      ; name of variable on the file
  fili    = "olr_daily_anomalies_1980-2018.nc"

  f       = addfile(diri+fili, "r")

  latS    = -60.            ; arbitrary
  latN    =  60.
  lonL    =  0.
  lonR    = 360.

  pltDir  = "./"                     ; plot directory
  pltType = "png"                    ; send graphics to PNG file
  pltName = "mjoclivar"

;************************************************************
; time indices corresponding to the desired time window
;************************************************************

  date  = cd_calendar(f->time, -2)   ; *entire* file
  iStrt = ind(date.eq.twStrt)        ; user specified dates
  iLast = ind(date.eq.twLast)
  delete(date)

;************************************************************
; Read user specified period
;************************************************************

;  if (getfilevartypes(f,vName) .eq. "short") then
;      X = short2flt( f->$vName$(iStrt:iLast,:,:) )
;  else
      X = f->$vName$(iStrt:iLast,:,:)
;  end if
printVarSummary( X )
;printMinMax(X, True)
 x       = X(:,{latS:latN},{lonL:lonR})   ; region

 printVarSummary( x )
 ;printMinMax(x, True)

 ;wgty    = latRegWgt(x&lat, "double", 0)

  dimx    = dimsizes( x )
  ntim    = dimx(0)

  date    = cd_calendar(x&time, -2)   ; selected period
  hStrt   = ind(date.eq.thStrt)       ; plot times
  hLast   = ind(date.eq.thLast)
  ntimp   = (hLast-hStrt) + 1

;************************************************
; Extract filtered series at each grid point
;************************************************

  optLatLon  = False
  mjo_latlon = band_pass_latlon_time (x, spd, bpf, optLatLon)

  printVarSummary(mjo_latlon)
  printMinMax(mjo_latlon, True)


;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;
 time = f->time(iStrt:iLast)
 lat = f->lat({latS:latN})
 lon = f->lon

;===================================================================  
; Assume variables T, PS and ORO exist and that they have 
; associated meta data: (a) coordinate variables time, lev, lat, lon       
; and (b) attributes
;===================================================================  
    ntim  = dimsizes(time)                 ; get dimension sizes                                                
nlat  = dimsizes(lat)  
nlon  = dimsizes(lon)      

    diro = "./"                     ; Output directory
    filo = "olr_20-100days_mjo_1980-2018.nc"             ; Output file
;system("/bin/rm -f " + diro + filo)    ; remove if exists
fout  = addfile (diro + filo, "c")  ; open output file

;===================================================================
; explicitly declare file definition mode. Improve efficiency.
;===================================================================
    setfileoption(fout,"DefineMode",True)

;===================================================================
; create global attributes of the file
;===================================================================
    fAtt               = True            ; assign file attributes
fAtt@title         = "NCL Efficient Approach to netCDF Creation"  
fAtt@source_file   =  "original-file.nc"
fAtt@Conventions   = "None"   
fAtt@creation_date = systemfunc ("date")        
fileattdef( fout, fAtt )            ; copy file attributes    

;===================================================================
; predefine the coordinate variables and their dimensionality
; Note: to get an UNLIMITED record dimension, we set the dimensionality
; to -1 (or the actual size) and set the dimension name to True.
;===================================================================
    dimNames = (/"time", "lat", "lon"/)  
dimSizes = (/ ntim ,  nlat,  nlon /) 
dimUnlim = (/False , False, False/)   
filedimdef(fout,dimNames,dimSizes,dimUnlim)

;===================================================================
; predefine the the dimensionality of the variables to be written out
;===================================================================
; Here we are using NCL functions to facilitate defining 
; each variable's dimension name(s) and type. 
; The following could be replaced with explicit, user defined dimension 
; names different from those associated with the variable in memory. 
; Say, PS(time,lat,lon) in the NCL script. They could be redefined for the file via: 
; filevardef(fout, "PS"   ,typeof(PS) ,(/"TIME","latitude","longitude"/)) 
;===================================================================
   filevardef(fout, "time" ,typeof(time),getvardims(time))                          
   filevardef(fout, "lat"  ,typeof(lat),getvardims(lat))                          
   filevardef(fout, "lon"  ,typeof(lon),getvardims(lon))                          
   filevardef(fout, "mjo_latlon" ,typeof(mjo_latlon)  ,getvardims(mjo_latlon))    

;===================================================================
; Copy attributes associated with each variable to the file
; All attributes associated with each variable will be copied.
;====================================================================

   filevarattdef(fout,"time" ,time)                    ; copy time attributes

   filevarattdef(fout,"lat"  ,lat)                     ; copy lat attributes
   filevarattdef(fout,"lon"  ,lon)                     ; copy lon attributes

   filevarattdef(fout,"mjo_latlon",mjo_latlon)                     ; copy TOPOG attributes

;===================================================================
; explicitly exit file definition mode. **NOT REQUIRED**
;===================================================================
    setfileoption(fout,"DefineMode",False)

;=======================http://localhost:8888/notebooks/Hendon1999_analysis.ipynb#============================================
; output only the data values since the dimensionality and such have
; been predefined. The "(/", "/)" syntax tells NCL to only output the
; data values to the predefined locations on the file.
;====================================================================
   fout->time   = (/time/)     
   fout->lat    = (/lat/)
   fout->lon    = (/lon/) 
   fout->mjo_latlon  = (/mjo_latlon/)

Variable: X
Type: float
Total Size: 598973760 bytes
            149743440 values
Number of Dimensions: 3
Dimensions and sizes:	[time | 14245] x [lat | 73] x [lon | 144]
Coordinates: 
            time: [1577832..1919688]
            lat: [90..-90]
            lon: [ 0..357.5]
Number Of Attributes: 12
  long_name :	Daily OLR
  units :	W/m^2
  _FillValue :	32766
  missing_value :	32766
  unpacked_valid_range :	(  0, 500 )
  actual_range :	( 77.25, 314 )
  var_desc :	Outgoing Longwave Radiation
  precision :	2
  dataset :	NOAA Interpolated OLR
  level_desc :	Other
  statistic :	Mean
  parent_stat :	Individual ObsVariable: x
Type: float
Total Size: 402050880 bytes
            100512720 values
Number of Dimensions: 3
Dimensions and sizes:	[time | 14245] x [lat | 49] x [lon | 144]
Coordinates: 
            time: [1577832..1919688]
            lat: [-60..60]
            lon: [ 0..357.5]
Number Of Attributes: 12
  parent_stat :	Individual Obs
  statistic :	Mean
  level_desc :	Other
  dataset :	